In [6]:
# load model_selection profile result
import glob
from collections import defaultdict
from benchmarking.constants import MODEL_COST
path = '../model_selection/results/'

modelselection_profile = defaultdict(list)
for file in glob.glob(path + '/*_profile_log_mergelabel.csv'):
    with open(file, 'r') as f:
        f.readline()
        for line in f:
            line_list = line.strip().split(',')
            key = line_list[0]
            f1 = float(line_list[2])
            gpu = float(MODEL_COST[line_list[1]])/MODEL_COST['FasterRCNN']
            modelselection_profile[key].append((gpu, f1))
            
        

In [46]:
# load noscope_selection profile result
path = '/mnt/data/zhujun/dataset/NoScope_finetuned_models/'
noscope_profile = defaultdict(list)
for file in glob.glob(path + '/*/data/tmp_log_overfitting_w_gpu_cost_min_gpu.csv'):
    with open(file, 'r') as f:
        f.readline()
        for line in f:
            line_list = line.strip().split(',')
            key = line_list[0]
            mse_thresh = float(line_list[1])
            if mse_thresh != 0:
                continue
            num_trigger_frame = len(line_list[4].split(' '))
            f1 = float(line_list[3])
            gpu = (MODEL_COST['mobilenet']*900 + MODEL_COST['FasterRCNN']*num_trigger_frame)/(900*MODEL_COST['FasterRCNN'])
            noscope_profile[key].append((gpu, f1))

In [50]:
from matplotlib import pyplot as plt
f1 = open('tradeoff_curve_NS.txt', 'w')
f2 = open('tradeoff_curve_MS.txt', 'w')
f1.write('#NS_gpu,NS_f1(using video crossroad3_7)\n')
f2.write('#MS_gpu,MS_f1(using video crossroad3_7)\n')

for (k, v) in modelselection_profile.items():
    if k not in noscope_profile:
        continue
    if 'crossroad3_7' not in k:
        continue

#     plt.plot([x[0] for x in v], [x[1] for x in v], c='b')
    if noscope_profile[k][0][1] < 0.7:
        continue
    print(k)
#     plt.plot([x[0] for x in noscope_profile[k]], [x[1] for x in noscope_profile[k]], c='r')
#     plt.xlim(0, 1.3)
#     plt.ylim(0, 1)
    for x in noscope_profile[k]:
        f1.write(str(x[0]) + ',' + str(x[1])+'\n')
    for x in modelselection_profile[k]:
        f2.write(str(x[0]) + ',' + str(x[1])+'\n')    
    
    


crossroad3_7
